<a href="https://colab.research.google.com/github/ajagota7/Shaping/blob/main/SCOPE_Lifegate_Run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# %cd /content/Shaping
# !git pull origin main

# Imports

In [2]:
import numpy as np
import os
from google.colab import drive
import pickle
# np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
from scipy.optimize import minimize
import random

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


import sys

import plotly.graph_objects as go
from plotly.subplots import make_subplots

import copy
from copy import deepcopy

from typing import List, Union, Callable

import pygame
import click
import random




pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Lifegate class play

In [3]:
# import os
# from copy import deepcopy
# import pygame
# import numpy as np
# import click


In [4]:
# # RGB colors
# WHITE = (255, 255, 255)
# BLACK = (0, 0, 0)
# RED = (255, 0, 0)
# BLUE = (0, 100, 255)
# GREEN = (0, 255, 0)
# WALL = (80, 80, 80)
# YELLOW = (255, 255, 0)



# class LifeGate(object):
#     def __init__(self, state_mode, rng, death_drag, max_steps=100, fixed_life=True, rendering=False, image_saving=False, render_dir=None):
#         self.rng = rng
#         self.state_dtype = np.float32
#         self.frame_skip = 1  # for env consistency
#         self.fixed_life = fixed_life
#         self.blue = BLUE
#         self.death_drag = death_drag
#         self.legal_actions = [0, 1, 2, 3, 4]
#         self.action_meanings = ['no-op', 'up', 'down', 'left', 'right']
#         self.reward_scheme = {'death': -1.0, 'recovery': +1.0, 'step': 0.0, 'barrier': 0.0}
#         self.nb_actions = len(self.legal_actions)
#         self.player_pos_x = None
#         self.player_pos_y = None
#         self.agent_init_pos = None
#         self.state_mode = state_mode    # how the returned state look like ('pixel' or '1hot' or 'multi-head')
#         # self.scr_w = None
#         # self.scr_h = None
#         # self.possible_recoveries = []
#         self.recovery_observablity = True
#         # self.observability_switch_point = None  # where to turn observability off
#         # self.rendering_scale = None
#         # self.barriers = None
#         self.recoveries = None
#         self.deaths = None
#         # self.dead_ends = None
#         self._rendering = rendering
#         # self.state_shape = None
#         self.init_subclass()
#         if rendering:
#             self._init_pygame()
#         self.image_saving = image_saving
#         self.render_dir_main = render_dir
#         self.render_dir = None
#         self.state = None
#         self.step_id = 0
#         self.game_over = False

#         self.max_steps = max_steps

#         self.reset()

#     def init_subclass(self):
#         # should implement sizes, barriers, recoveries, deaths, init_player(), and rendering_scale
#         self.scr_w, self.scr_h = 10, 10
#         self.tabular_state_shape = (self.scr_w, self.scr_h)
#         self.state_shape = [24]
#         self.rendering_scale = 30
#         self.barriers = [[0, 0], [1, 0], [2, 0], [3, 0], [4, 0], [1, 5], [2, 5], [3, 5], [4, 5]]
#         self.possible_recoveries = [[5, 0], [6, 0], [7, 0]]
#         self.main_deaths = [[self.scr_w - 1, k] for k in range(self.scr_h)] + [[8,0]]
#         self.dead_ends = [[x, y] for x in range(self.scr_w // 2, self.scr_w - 1) for y in range(self.scr_w // 2, self.scr_w)]
#         self.observability_switch_point = [0, 5]

#     @property
#     def rendering(self):
#         return self._rendering

#     @rendering.setter
#     def rendering(self, flag):
#         if flag is True:
#             if self._rendering is False:
#                 self._init_pygame()
#                 self._rendering = True
#         else:
#             self.close()
#             self._rendering = False

#     def _init_pygame(self):
#         pygame.init()
#         size = [self.rendering_scale * self.scr_w, self.rendering_scale * self.scr_h]
#         self.screen = pygame.display.set_mode(size)
#         pygame.display.set_caption("LifeGate")

#     def _init_rendering_folder(self):
#         if self.render_dir_main is None:
#             self.render_dir_main = 'render'
#         if not os.path.exists(os.path.join(os.getcwd(), self.render_dir_main)):
#             os.mkdir(os.path.join(os.getcwd(), self.render_dir_main))
#         i = 0
#         while os.path.exists(os.path.join(os.getcwd(), self.render_dir_main, 'render' + str(i))):
#             i += 1
#         self.render_dir = os.path.join(os.getcwd(), self.render_dir_main, 'render' + str(i))
#         os.mkdir(self.render_dir)

#     def reset(self):
#         if self.image_saving:
#             self._init_rendering_folder()
#         self.game_over = False
#         self.step_id = 0
#         self.recovery_observablity = True
#         self.blue = BLUE
#         state = self.init_episode()
#         return state

#     def init_episode(self):
#         # should implement reconfigurations at the beginning of each episode
#         self.player_pos_x, self.player_pos_y = 2, self.scr_h - 1
#         targets = deepcopy(self.possible_recoveries)
#         # if self.fixed_life == True:
#         #     rec = targets.pop(2)  # fixed life-gate for DQN
#         # else:
#         #     rec = targets.pop(self.rng.randint(len(targets)))
#         self.recoveries = targets #[rec]
#         self.deaths = self.main_deaths #+ targets
#         return self.get_obs(self.state_mode)

#     def render(self):
#         if not self.rendering:
#             return
#         pygame.event.pump()
#         self.screen.fill(BLACK)
#         size = [self.rendering_scale, self.rendering_scale]
#         for pos in self.dead_ends:
#             p = [self.rendering_scale * pos[0], self.rendering_scale * pos[1]]
#             rec1 = pygame.Rect(p[0], p[1], size[0], size[1])
#             pygame.draw.rect(self.screen, YELLOW, rec1)
#         player = pygame.Rect(self.rendering_scale * self.player_pos_x, self.rendering_scale * self.player_pos_y,
#                              size[0], size[1])
#         pygame.draw.rect(self.screen, WHITE, player)
#         for pos in self.deaths:
#             p = [self.rendering_scale * pos[0], self.rendering_scale * pos[1]]
#             rec1 = pygame.Rect(p[0], p[1], size[0], size[1])
#             pygame.draw.rect(self.screen, RED, rec1)
#         for pos in self.recoveries:
#             p = [self.rendering_scale * pos[0], self.rendering_scale * pos[1]]
#             rec1 = pygame.Rect(p[0], p[1], size[0], size[1])
#             pygame.draw.rect(self.screen, self.blue, rec1)  # self.blue will change if reach obs point
#         for pos in self.barriers:
#             p = [self.rendering_scale * pos[0], self.rendering_scale * pos[1]]
#             rec1 = pygame.Rect(p[0], p[1], size[0], size[1])
#             pygame.draw.rect(self.screen, WALL, rec1)
#         pygame.display.flip()

#         if self.image_saving:
#             self.save_image()

#     def save_image(self):
#         if self.rendering and self.render_dir is not None:
#             pygame.image.save(self.screen, self.render_dir + '/render' + str(self.step_id) + '.jpg')
#         else:
#             raise ValueError('env.rendering is False and/or environment has not been reset.')

#     def close(self):
#         if self.rendering:
#             pygame.quit()

#     def _move_player(self, action):
#         x, y = (self.player_pos_x, self.player_pos_y)
#         # dead-end:
#         if [x, y] in self.dead_ends:
#             if self.rng.binomial(1, 0.70):
#                 action = 4  # forceful right
#             else:
#                 action = 0  # no-op
#         else:
#             # natural risk of death
#             if self.rng.binomial(1, self.death_drag):  # say with 25% if death_drag==0.25
#                 action = 4

#         if action == 4:    # right
#             x += 1
#         elif action == 3:  # left
#             x -= 1
#         elif action == 2:  # down
#             y += 1
#         elif action == 1:  # up
#             y -= 1
#         # updating the position
#         if [x, y] in self.barriers or x < 0 or y < 0 or y >= self.scr_h:
#             return
#         else:
#             self.player_pos_x, self.player_pos_y = x, y

#     def _get_status(self):
#         # check the current situation
#         if [self.player_pos_x, self.player_pos_y] in self.deaths:
#             return 'death'
#         elif [self.player_pos_x, self.player_pos_y] in self.recoveries:
#             return 'recovery'

#     def step(self, action):
#         assert action in self.legal_actions, 'Illegal action.'
#         if self.step_id >= self.max_steps - 1:
#             self.game_over = True
#             return self.get_obs(self.state_mode), 0., self.game_over, {}
#         self.step_id += 1
#         self._move_player(action)
#         if [self.player_pos_x, self.player_pos_y] == self.observability_switch_point and self.recovery_observablity == True:
#             self.recovery_observablity = False
#             self.blue = BLACK
#         status = self._get_status()
#         if status == 'death':
#             self.game_over = True
#             reward = self.reward_scheme['death']
#         elif status == 'recovery':
#             self.game_over = True
#             reward = self.reward_scheme['recovery']
#         else:
#             reward = self.reward_scheme['step']
#         return self.get_obs(self.state_mode), reward, self.game_over, {}

#     def get_lives(self):
#         if self.game_over == True:
#             return 0
#         else:
#             return 1

#     def get_state(self):
#         return self.get_obs(self.state_mode)

#     def get_obs(self, method):
#         if method == 'vector':
#             return self._get_vec_obs()
#         elif method == 'pixel':
#             return self._get_pixel_obs()
#         elif method == 'tabular':
#             return self._get_tabular_obs()
#         else:
#             raise ValueError('Unknown observation method.')

#     def _get_vec_obs(self):
#         x = np.zeros(self.scr_w + self.scr_h + len(self.possible_recoveries), dtype=self.state_dtype)
#         x[self.player_pos_x] = 1.0
#         x[self.player_pos_y + self.scr_w] = 1.0
#         if self.recovery_observablity == True or self.fixed_life == True:
#             for k in self.recoveries:
#                 x[k[0] - 5 + self.scr_w + self.scr_h] = 1.0
#         return x

#     def _get_tabular_obs(self):
#         return np.array([self.player_pos_x, self.player_pos_y])

#     def _get_pixel_obs(self):
#         raise NotImplementedError

# shaping dependencies

In [5]:
!git clone https://github.com/ajagota7/Shaping.git

Cloning into 'Shaping'...
remote: Enumerating objects: 319, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 319 (delta 63), reused 76 (delta 33), pack-reused 212
Receiving objects: 100% (319/319), 12.99 MiB | 7.56 MiB/s, done.
Resolving deltas: 100% (178/178), done.


In [6]:
import zipfile

with zipfile.ZipFile('/content/Shaping/lifegate_1.zip', 'r') as zip_ref:
    # zip_ref.extractall('/content/med-deadend/lifegate/results/lifegate_1')
    zip_ref.extractall('/content/Shaping/')

In [7]:
import sys
sys.path.append('/content/Shaping/')



In [8]:
import q_networks

In [9]:
# %cd /content/med-deadend/lifegate


# results_dir = 'results/lifegate_1/'
results_dir = '/content/Shaping/'
# Load the Q tables from the primary learning agent, Q_D and Q_R value functions
with open(results_dir+'tabular_qnet.pkl', 'rb') as fq:
    ai = pickle.load(fq)

with open(results_dir+'tabular_qd.pkl', 'rb') as fd:
    ai_d = pickle.load(fd)

with open(results_dir+'tabular_qr.pkl', 'rb') as fr:
    ai_r = pickle.load(fr)

In [10]:
q_table = np.zeros((10, 10, 5))
q_d = np.zeros_like(q_table)
q_r = np.zeros_like(q_table)


for i in range(10):
    for j in range(10):
        for a in range(5):
            key = tuple([j, i, a])
            try:
                q_table[i,j,a] = ai.q[key]
                q_d[i,j,a] = ai_d.q[key]
                q_r[i,j,a] = ai_r.q[key]
            except:
                pass

In [11]:
q_table[0][0]

array([0., 0., 0., 0., 0.])

In [12]:
# import yaml
# import random
# from lifegate import LifeGate
# params = yaml.safe_load(open(results_dir+'config.yaml', 'r'))
# np.random.seed(seed=params['random_seed'])
# random.seed(params['random_seed'])
# random_state = np.random.RandomState(params['random_seed'])

In [13]:
import Shaping
from Shaping import *
from shaping_features import *
from IS import *
from neural_net import *
from SCOPE_straight import SCOPE_straight
from existing_experiments import existing_experiments
from SCOPE_experiment import SCOPE_experiment
from multi_experiment_visualisations import *
from lifegate import LifeGate

# Mount Drive

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Varying Shaping Function Sets

The following experiments are organized by shaping function. Given a chosen shaping function, the number of trajectories, max length of trajectories, neural network architecture, and train set size among other variables are played with

## smallest distance to dead end

In [ ]:
params = {
    # Parameters related to policy generation
    "pi_b_top_k": 1,
    "pi_b_epsilon": 0.4,
    "pi_e_top_k": 1,
    "pi_e_epsilon": 0.05,
    "q_table": q_table,
    "gamma": 0.99,
    "num_trajectories": 200,
    "num_bootstraps": 10000,
    "percent_to_estimate_phi": 0.3,

    # Parameters related to shaping
    "shaping_feature": smallest_distance_to_deadend,
    "shaping_coefficient": 0.1,

    # Parameters related to neural network architecture and training
    "hidden_dims": [8, 8],
    "learning_rate": 0.001,
    "dropout_prob": 0.2,
    "l1_reg": 0.00001,
    "l2_reg": 0.00001,
    "scope_weight": 1,
    "mse_weight": 1,
    "num_epochs": 300,

    # Parameters related to environment
    "max_length": 50,
    "death_drag": 0.0,
    # Other general parameters
    "dtype": torch.float64,
    "experiment_type": "test",
    "folder_path": "/content/drive/MyDrive/Lifegate_experiments"
    # "folder_path": "/content"
}

In [ ]:
# Define the varying trajectory lengths
num_trajectories = [200, 400, 600, 800, 1000]
# Define hidden_dims
hidden_dimens = [[8],[8,8],[32]]
# Define trajectory lengths
trajectory_length = [30, 50, 70, 100]

for num in num_trajectories:
  # Modify the base parameters for each experiment
  # params = base_params_test.copy()  # Make a copy to avoid modifying the base parameters
  params["num_trajectories"] = num  # Update the number of trajectories

  for dims in hidden_dimens:
    # params = base_params_test.copy()  # Make a copy to avoid modifying the base parameters
    params["hidden_dims"] = dims

    for len in trajectory_length:
      params["max_length"] = len

      # Create an instance of SCOPE_experiment with modified parameters
      test_experiment = SCOPE_experiment(**params)
      test_experiment.run_experiment()

In [ ]:
# Define the varying trajectory lengths
num_trajectories = [200, 400, 600, 800, 1000]
# Define hidden_dims
hidden_dimens = [[8],[8,8],[32]]
# Define trajectory lengths
trajectory_length = [30, 50, 70, 100]

for num in num_trajectories:
  # Modify the base parameters for each experiment
  # params = base_params_test.copy()  # Make a copy to avoid modifying the base parameters
  params["num_trajectories"] = num  # Update the number of trajectories

  for dims in hidden_dimens:
    # params = base_params_test.copy()  # Make a copy to avoid modifying the base parameters
    params["hidden_dims"] = dims

    for len in trajectory_length:
      params["max_length"] = len

      # Create an instance of SCOPE_experiment with modified parameters
      test_experiment = SCOPE_experiment(**params)
      # test_experiment.run_experiment()
      test_experiment.plot_metrics_save(save= True)

## smallest distance to recovery

In [ ]:
params = {
    # Parameters related to policy generation
    "pi_b_top_k": 1,
    "pi_b_epsilon": 0.4,
    "pi_e_top_k": 1,
    "pi_e_epsilon": 0.05,
    "q_table": q_table,
    "gamma": 0.99,
    "num_trajectories": 200,
    "num_bootstraps": 10000,
    "percent_to_estimate_phi": 0.3,

    # Parameters related to shaping
    "shaping_feature": smallest_distance_to_recovery,
    "shaping_coefficient": 0.1,

    # Parameters related to neural network architecture and training
    "hidden_dims": [8],
    "learning_rate": 0.001,
    "dropout_prob": 0.2,
    "l1_reg": 0.00001,
    "l2_reg": 0.00001,
    "scope_weight": 1,
    "mse_weight": 1,
    "num_epochs": 300,

    # Parameters related to environment
    "max_length": 30,
    "death_drag": 0.0,
    # Other general parameters
    "dtype": torch.float64,
    "experiment_type": "test",
    "folder_path": "/content/drive/MyDrive/Lifegate_experiments"
    # "folder_path": "/content"
}

In [ ]:
test_experiment = SCOPE_experiment(**params)
# test_experiment.run_experiment()
test_load = existing_experiments(test_experiment)
test_load.get_heatmap()

In [ ]:
# Define the varying trajectory lengths
num_trajectories = [200, 400, 600, 800, 1000]
# Define hidden_dims
hidden_dimens = [[8],[8,8],[32]]
# Define trajectory lengths
trajectory_length = [30, 50, 70, 100]

for num in num_trajectories:
  # Modify the base parameters for each experiment
  # params = base_params_test.copy()  # Make a copy to avoid modifying the base parameters
  params["num_trajectories"] = num  # Update the number of trajectories

  for dims in hidden_dimens:
    # params = base_params_test.copy()  # Make a copy to avoid modifying the base parameters
    params["hidden_dims"] = dims

    for len in trajectory_length:
      params["max_length"] = len

      # Create an instance of SCOPE_experiment with modified parameters
      test_experiment = SCOPE_experiment(**params)
      test_experiment.run_experiment()

In [ ]:
# Define the varying trajectory lengths
num_trajectories = [200, 400, 600, 800, 1000]
# Define hidden_dims
hidden_dimens = [[8],[8,8],[32]]
# Define trajectory lengths
trajectory_length = [30, 50, 70, 100]

for num in num_trajectories:
  # Modify the base parameters for each experiment
  # params = base_params_test.copy()  # Make a copy to avoid modifying the base parameters
  params["num_trajectories"] = num  # Update the number of trajectories

  for dims in hidden_dimens:
    # params = base_params_test.copy()  # Make a copy to avoid modifying the base parameters
    params["hidden_dims"] = dims

    for len in trajectory_length:
      params["max_length"] = len

      # Create an instance of SCOPE_experiment with modified parameters
      test_experiment = SCOPE_experiment(**params)
      # test_experiment.run_experiment()
      test_load = existing_experiments(test_experiment)
      test_load.plot_metrics_save(save= True)
      print(test_load.experiment_instance.generate_file_name())


In [ ]:
# Define the varying trajectory lengths
num_trajectories = [200, 400, 600, 800, 1000]
# Define hidden_dims
hidden_dimens = [[8],[8,8],[32]]
# Define trajectory lengths
trajectory_length = [30, 50, 70, 100]

for num in num_trajectories:
  # Modify the base parameters for each experiment
  # params = base_params_test.copy()  # Make a copy to avoid modifying the base parameters
  params["num_trajectories"] = num  # Update the number of trajectories

  for dims in hidden_dimens:
    # params = base_params_test.copy()  # Make a copy to avoid modifying the base parameters
    params["hidden_dims"] = dims

    for len in trajectory_length:
      params["max_length"] = len

      # Create an instance of SCOPE_experiment with modified parameters
      test_experiment = SCOPE_experiment(**params)
      # test_experiment.run_experiment()
      test_load = existing_experiments(test_experiment)
      test_load.plot_metrics_save(save= True)
      print(test_load.experiment_instance.generate_file_name())

In [ ]:
num_trajectories = [200, 400, 600, 800, 1000]
dims = [[8], [8,8], [32]]
trajectory_length = [30, 50, 70, 100]

for i in dims:
  params["hidden_dims"] = i
  for len in trajectory_length:
    params["max_length"] = len
    save__experiments_over_trajectories(params, num_trajectories)

## Regional Shaping around bottleneck

In [15]:
params = {
    # Parameters related to policy generation
    "pi_b_top_k": 1,
    "pi_b_epsilon": 0.7,
    "pi_e_top_k": 1,
    "pi_e_epsilon": 0.2,
    "q_table": q_table,
    "gamma": 0.99,
    "num_trajectories": 800,
    "num_bootstraps": 10000,
    "percent_to_estimate_phi": 0.3,

    # Parameters related to shaping
    "shaping_feature": bottleneck_four_regions_k_p9_a_1,
    "shaping_coefficient": 0.1,

    # Parameters related to neural network architecture and training
    "hidden_dims": [8,8],
    "learning_rate": 0.001,
    "dropout_prob": 0.2,
    "l1_reg": 0.00001,
    "l2_reg": 0.00001,
    "scope_weight": 1.0,
    "mse_weight": 1.0,
    "num_epochs": 500,

    # Parameters related to environment
    "max_length": 50,
    "death_drag": 0.0,
    # Other general parameters
    "dtype": torch.float32,
    "experiment_type": "test",
    "folder_path": "/content/drive/MyDrive/Lifegate_experiments"
    # "folder_path": "/content"
}

In [17]:
list_trajectories = [200, 400, 600, 800, 1000]

for i in list_trajectories:
  params["num_trajectories"] = i
  test_experiment = SCOPE_experiment(**params)
  print(f"{i} trajectories:")

  test_load = existing_experiments(test_experiment)
  test_load.plot_metrics()
  test_load.get_heatmap()
  test_load.get_state_visitation_heatmap()

  # test_experiment.run_experiment()

200 trajectories:


400 trajectories:


600 trajectories:


800 trajectories:


1000 trajectories:


In [18]:
num_trajectories = [200, 400, 600, 800, 1000]
viz_over_num_trajectories(params, num_trajectories)

In [41]:
# max_length: 70

list_trajectories = [200, 400, 600, 800, 1000]

for i in list_trajectories:
  params["num_trajectories"] = i
  test_experiment = SCOPE_experiment(**params)
  print(f"{i} trajectories:")
  test_load = existing_experiments(test_experiment)
  test_load.plot_metrics()
  test_load.get_heatmap()
  test_load.get_state_visitation_heatmap()
  # test_experiment.run_experiment()


# test_experiment = SCOPE_experiment(**params)
# test_experiment.run_experiment()
# test_experiment.continue_training(300)
# test_load = existing_experiments(test_experiment)

200 trajectories:


400 trajectories:


600 trajectories:


800 trajectories:


1000 trajectories:


In [17]:
num_trajectories = [200, 400, 600, 800, 1000]
viz_over_num_trajectories(params, num_trajectories)

In [ ]:
pi_b = test_load.load_pi_b()

In [ ]:
test_load = existing_experiments(test_experiment)
test_load.plot_metrics()

In [ ]:
test_load.get_state_visitation_heatmap()

In [ ]:
test_load.get_heatmap()

In [ ]:
test_load = existing_experiments(test_experiment)
test_load.plot_metrics()

In [ ]:
test_experiment.dtype

torch.float32

In [ ]:
test_load.get_heatmap()

## Regional shaping around bottleneck v2

Differ folder path with names incorporating policy parameters num top actions and epsilon of $\pi_e$ and $\pi_b$

In [15]:
params = {
    # Parameters related to policy generation
    "pi_b_top_k": 1,
    "pi_b_epsilon": 0.6,
    "pi_e_top_k": 1,
    "pi_e_epsilon": 0.1,
    "q_table": q_table,
    "gamma": 0.99,
    "num_trajectories": 800,
    "num_bootstraps": 10000,
    "percent_to_estimate_phi": 0.3,

    # Parameters related to shaping
    "shaping_feature": bottleneck_four_regions_k_p9_a_1,
    "shaping_coefficient": 0.1,

    # Parameters related to neural network architecture and training
    "hidden_dims": [8,8],
    "learning_rate": 0.001,
    "dropout_prob": 0.2,
    "l1_reg": 0.00001,
    "l2_reg": 0.00001,
    "scope_weight": 1.0,
    "mse_weight": 1.0,
    "num_epochs": 500,

    # Parameters related to environment
    "max_length": 50,
    "death_drag": 0.0,
    # Other general parameters
    "dtype": torch.float32,
    "experiment_type": "test",
    "folder_path": "/content/drive/MyDrive/Lifegate_experiments_v2"
    # "folder_path": "/content"
}

In [ ]:
list_trajectories = [200, 400, 600, 800, 1000]

for i in list_trajectories:
  params["num_trajectories"] = i
  test_experiment = SCOPE_experiment(**params)
  print(f"{i} trajectories:")

  # test_load = existing_experiments(test_experiment)
  # test_load.plot_metrics()
  # test_load.get_heatmap()
  # test_load.get_state_visitation_heatmap()

  test_experiment.run_experiment()

200 trajectories:


/content/Shaping/SCOPE_straight.py:115: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  padded_timestep_tensors = torch.tensor(padded_timesteps, dtype = self.dtype)


Streaming output truncated to the last 5000 lines.
Epoch 351
IS mean: 0.024885760620236397,IS variance: 0.00014304844080470502
SCOPE Var loss:  tensor(0.0151, grad_fn=<VarBackward0>)
MSE loss:  0.12636297941207886
SCOPE mean: 0.07641627639532089, SCOPE var: 0.000325129134580493
Total Loss: 0.1415061354637146
----------------------------------------
Epoch 352
IS mean: 0.024885760620236397,IS variance: 0.00014304844080470502
SCOPE Var loss:  tensor(0.0150, grad_fn=<VarBackward0>)
MSE loss:  0.12595312297344208
SCOPE mean: 0.07636675238609314, SCOPE var: 0.00032408753759227693
Total Loss: 0.1409413367509842
----------------------------------------
Epoch 353
IS mean: 0.024885760620236397,IS variance: 0.00014304844080470502
SCOPE Var loss:  tensor(0.0148, grad_fn=<VarBackward0>)
MSE loss:  0.12554620206356049
SCOPE mean: 0.07631754130125046, SCOPE var: 0.00032306142384186387
Total Loss: 0.14038123190402985
----------------------------------------
Epoch 354
IS mean: 0.024885760620236397,IS v

# Testing IS

In [22]:
params = {
    # Parameters related to policy generation
    "pi_b_top_k": 1,
    "pi_b_epsilon": 0.01,
    "pi_e_top_k": 1,
    "pi_e_epsilon": 0.2,
    "q_table": q_table,
    "gamma": 0.99,
    "num_trajectories": 400,
    "num_bootstraps": 10000,
    "percent_to_estimate_phi": 0.3,

    # Parameters related to shaping
    "shaping_feature": bottleneck_four_regions_k_p9_a_1,
    "shaping_coefficient": 0.1,

    # Parameters related to neural network architecture and training
    "hidden_dims": [8,8],
    "learning_rate": 0.001,
    "dropout_prob": 0.2,
    "l1_reg": 0.00001,
    "l2_reg": 0.00001,
    "scope_weight": 1.0,
    "mse_weight": 1.0,
    "num_epochs": 500,

    # Parameters related to environment
    "max_length": 70,
    "death_drag": 0.0,
    # Other general parameters
    "dtype": torch.float32,
    "experiment_type": "test",
    "folder_path": "/content/drive/MyDrive/Lifegate_experiments"
    # "folder_path": "/content"
}

In [20]:
test_experiment = SCOPE_experiment(**params)

In [58]:
# test_load = existing_experiments(test_experiment)
# test_load.plot_metrics()

In [59]:
# pi_b = test_load.load_pi_b()

In [62]:
pi_b_run, pi_e, model, experiment_class = test_experiment.prepare_experiment()

In [21]:
def get_rho(trajectories, P_pi_b, P_pi_e):
    episode_rhos = []
    episode_rews = []
    episode_probs_b_and_e = []  # List to store (P_e, P_b) tuples

    for i in range(len(trajectories)):
        rho = []  # Initialize rho for the current episode
        rew = []
        probs_b_and_e = []  # Initialize (P_e, P_b) list for the current episode

        for j in range(len(trajectories[i]['state'])):
            state = trajectories[i]['state'][j]
            action = trajectories[i]['action'][j]
            reward = trajectories[i]['reward'][j]

            P_e = P_pi_e[int(state[0])][int(state[1])][action]
            P_b = P_pi_b[int(state[0])][int(state[1])][action]

            rho.append(P_e / P_b)
            rew.append(reward)
            probs_b_and_e.append((state.tolist(), action, P_e, P_b, P_e/P_b))

        episode_rews.append(rew)
        episode_rhos.append(rho)
        episode_probs_b_and_e.append(probs_b_and_e)

    return episode_rhos, episode_rews, episode_probs_b_and_e

def step_IS(episode_rhos, episode_rews, return_Qs=False):

  V_step_IS = [np.sum(0.99**np.arange(len(rews)) * np.cumprod(np.array(rhos)) * np.array(rews))  for rhos, rews in zip(episode_rhos, episode_rews)]
  if return_Qs:
      return np.mean(V_step_IS), np.array(V_step_IS)
  else:
      return np.mean(V_step_IS)

def calc_var_bootstrap(all_V_step_IS):
  # Parameters
  n_bootstrap_samples = 30000  # Number of bootstrap samples to generate
  n_elements = len(all_V_step_IS)  # Number of elements in the original array

  # Bootstrap sampling and calculation of means
  bootstrap_means = np.empty(n_bootstrap_samples)

  for i in range(n_bootstrap_samples):
      bootstrap_sample = np.random.choice(all_V_step_IS, size=n_elements, replace=True)
      bootstrap_means[i] = np.mean(bootstrap_sample)

  # Calculate the variance of the bootstrap sample means
  variance_of_means = np.var(bootstrap_means)

  print(f"Variance of stepIS: {variance_of_means}")
  return variance_of_means, bootstrap_means

def PDIS(params):
  test_experiment = SCOPE_experiment(**params)
  pi_b_run, pi_e, model, experiment_class = test_experiment.prepare_experiment()
  P_pi_b =  test_experiment.action_probs_top_n_epsilon(params["pi_b_top_k"],params["pi_b_epsilon"])
  P_pi_e =  test_experiment.action_probs_top_n_epsilon(params["pi_e_top_k"],params["pi_e_epsilon"])
  episode_rhos, episode_rews, episode_probs_b_and_e = get_rho(pi_b_run,P_pi_b, P_pi_e)
  V_step_IS, all_V_step_IS = step_IS(episode_rhos, episode_rews, return_Qs=True)
  print(f"Value estimate of stepIS: {V_step_IS}")
  variance_of_means, bootstrap_means = calc_var_bootstrap(all_V_step_IS)
  on_policy = experiment_class.calc_V_pi_e()
  bias = V_step_IS - on_policy
  mse = variance_of_means + bias**2
  print(f"On Policy Estimate: {on_policy}")
  print(f"MSE: {mse}")
  # return V_step_IS, variance_of_means, on_policy, mse


In [52]:
params["num_trajectories"] = 200000
PDIS(params)

Value estimate of stepIS: 0.8305026161037643
Variance of stepIS: 0.01484951434268996


In [127]:
# pi_b_load = test_load.load_pi_b()

In [46]:
params["pi_b_epsilon"] = 0.7
params["pi_e_epsilon"] = 0.2
params["num_trajectories"] = 200
params["pi_b_top_k"] = 1
params["pi_e_top_k"] = 1
params["max_length"] = 50

test_experiment = SCOPE_experiment(**params)

P_pi_b =  test_experiment.action_probs_top_n_epsilon(params["pi_b_top_k"],params["pi_b_epsilon"])
P_pi_e =  test_experiment.action_probs_top_n_epsilon(params["pi_e_top_k"],params["pi_e_epsilon"])



In [47]:
num_trajectories = [200, 400, 600, 800, 1000, 2000, 10000, 20000]

for i in num_trajectories:
  params["num_trajectories"] = i
  print(f"stepIS for {i} trajectories:")
  test_experiment = SCOPE_experiment(**params)
  pi_b_run, pi_e, model, experiment_class = test_experiment.prepare_experiment()
  episode_rhos, episode_rews, episode_probs_b_and_e = get_rho(pi_b_run,P_pi_b, P_pi_e)
  V_step_IS, all_V_step_IS = step_IS(episode_rhos, episode_rews, return_Qs=True)
  print(f"Value estimate of stepIS: {V_step_IS}")
  variance_of_means, bootstrap_means = calc_var_bootstrap(all_V_step_IS)
  print("-"*60)

stepIS for 200 trajectories:
Value estimate of stepIS: 0.029821977525163846
Variance of stepIS: 0.0004054829883995591
------------------------------------------------------------
stepIS for 400 trajectories:
Value estimate of stepIS: 0.04527415996475298
Variance of stepIS: 0.0005281308205243704
------------------------------------------------------------
stepIS for 600 trajectories:
Value estimate of stepIS: 0.03327212328286666
Variance of stepIS: 0.00023735137630116691
------------------------------------------------------------
stepIS for 800 trajectories:
Value estimate of stepIS: 0.08724145638248061
Variance of stepIS: 0.0015746900495902793
------------------------------------------------------------
stepIS for 1000 trajectories:
Value estimate of stepIS: 0.24904094324123893
Variance of stepIS: 0.03159243836446429
------------------------------------------------------------
stepIS for 2000 trajectories:
Value estimate of stepIS: 0.23594812198374696
Variance of stepIS: 0.01213343504

In [23]:
num_trajectories = [200, 400, 600, 800, 1000, 2000, 10000, 20000]
params["pi_b_epsilon"] = 0.7
params["pi_e_epsilon"] = 0.2
params["max_length"] = 70


for i in num_trajectories:
  params["num_trajectories"] = i
  print(f"stepIS for {i} trajectories:")
  PDIS(params)
  print("-"*60)


stepIS for 200 trajectories:
Value estimate of stepIS: 0.04058253983072969
Variance of stepIS: 0.0013015991629474674
On Policy Estimate: 0.8210778338655641
MSE: 0.6104745031734701
------------------------------------------------------------
stepIS for 400 trajectories:
Value estimate of stepIS: 0.04056869655732113
Variance of stepIS: 0.0006609250382466745
On Policy Estimate: 0.8214166746973824
MSE: 0.6103844900036683
------------------------------------------------------------
stepIS for 600 trajectories:
Value estimate of stepIS: 1.9257653531359342
Variance of stepIS: 3.5615824580710633
On Policy Estimate: 0.8194442378007352
MSE: 4.785528868307582
------------------------------------------------------------
stepIS for 800 trajectories:
Value estimate of stepIS: 1.675742483891416
Variance of stepIS: 2.0617370336993903
On Policy Estimate: 0.818598120170397
MSE: 2.7964334939581006
------------------------------------------------------------
stepIS for 1000 trajectories:
Value estimate of

In [62]:
num_trajectories = [200, 400, 600, 800, 1000, 2000, 10000, 20000]
params["pi_b_epsilon"] = 0.4
params["pi_e_epsilon"] = 0.1


for i in num_trajectories:
  params["num_trajectories"] = i
  print(f"stepIS for {i} trajectories:")
  PDIS(params)
  print("-"*60)




stepIS for 200 trajectories:
Value estimate of stepIS: 0.7284906335294948
Variance of stepIS: 0.07393974464103817
On Policy Estimate: 0.8417444727027873
MSE: 0.08676617672852815
------------------------------------------------------------
stepIS for 400 trajectories:
Value estimate of stepIS: 0.754259635576002
Variance of stepIS: 0.033584967453206896
On Policy Estimate: 0.8415875713420845
MSE: 0.041211135818371936
------------------------------------------------------------
stepIS for 600 trajectories:
Value estimate of stepIS: 0.6785665894269916
Variance of stepIS: 0.020293800503219637
On Policy Estimate: 0.8420081631662425
MSE: 0.04700694852958264
------------------------------------------------------------
stepIS for 800 trajectories:
Value estimate of stepIS: 0.6784946549232012
Variance of stepIS: 0.016145375344962683
On Policy Estimate: 0.8416197230091972
MSE: 0.042755163183023526
------------------------------------------------------------
stepIS for 1000 trajectories:
Value esti

In [63]:
num_trajectories = [200, 400, 600, 800, 1000, 2000, 10000, 20000, 50000]
params["pi_b_epsilon"] = 0.5
params["pi_e_epsilon"] = 0.2


for i in num_trajectories:
  params["num_trajectories"] = i
  print(f"stepIS for {i} trajectories:")
  PDIS(params)
  print("-"*60)

stepIS for 200 trajectories:
Value estimate of stepIS: 1.3739434596319946
Variance of stepIS: 0.35499455848865225
On Policy Estimate: 0.8200626355198506
MSE: 0.6617785258078
------------------------------------------------------------
stepIS for 400 trajectories:
Value estimate of stepIS: 1.241113034486646
Variance of stepIS: 0.1743812493136254
On Policy Estimate: 0.8213999703006682
MSE: 0.350540305562008
------------------------------------------------------------
stepIS for 600 trajectories:
Value estimate of stepIS: 0.9008434635294275
Variance of stepIS: 0.07975743285180345
On Policy Estimate: 0.8214505599982884
MSE: 0.0860606659829082
------------------------------------------------------------
stepIS for 800 trajectories:
Value estimate of stepIS: 0.9433142197398461
Variance of stepIS: 0.05028511794907695
On Policy Estimate: 0.8210499544690547
MSE: 0.0652336685112834
------------------------------------------------------------
stepIS for 1000 trajectories:
Value estimate of stepIS

In [59]:
params["num_trajectories"] = 200
test_experiment = SCOPE_experiment(**params)
pi_b_run, pi_e, model, experiment_class = test_experiment.prepare_experiment()

In [60]:
experiment_class.calc_V_pi_e()

0.8417444727027873

In [178]:
pi_e  = test_experiment.experiment_actions(10000, test_experiment.initialize_env(),P_pi_e)

In [181]:
def calc_V_pi_e(pi_e):
      all_timesteps = []
      gamma = test_experiment.gamma
      for j in range(len(pi_e)):
          Timestep_values = []
          for i in range(len(pi_e[j])):
            # print(i)
            timestep = gamma ** (i) * pi_e[j][i][2]
            Timestep_values.append(timestep)

          all_timesteps.append(Timestep_values)

      V_est = sum([sum(sublist) for sublist in all_timesteps])/len(pi_e)
      return V_est

In [182]:
calc_V_pi_e(pi_e)

0.8191611431930346

In [122]:
# SC_straight = SCOPE_straight(model = None, gamma = test_experiment.gamma, num_bootstraps=30000, pi_b = pi_b_run, P_pi_b = P_pi_b, pi_e = pi_e, P_pi_e = P_pi_e, percent_to_estimate_phi = 0.3, shaping_function = smallest_distance_to_death, env = test_experiment.initialize_env(), dtype = None)

In [123]:
# SC_straight.IS_pipeline()

In [125]:
np.var(all_V_step_IS)

0.7917807934639121

In [106]:
# Calculate the min and max values
min_value = np.min(bootstrap_means)
max_value = np.max(bootstrap_means)

# Normalize the array
normalized_bootstrap_means = (bootstrap_means - min_value) / (max_value - min_value)

variance = np.var(normalized_bootstrap_means)
print(variance)

0.019792605354133245


In [108]:
scaled_bootstrap_means = bootstrap_means * 1e2  # Example scaling factor
variance = np.var(scaled_bootstrap_means)
print(variance)


209.1000522177498


In [109]:
mad = np.median(np.abs(bootstrap_means - np.median(bootstrap_means)))
variance = mad * mad  # Variance approximation using MAD
print(variance)

0.009898897176736901


In [110]:
variance = np.var(bootstrap_means, dtype=np.float64)
print(variance)

0.020910005221774983


In [113]:
# log_bootstrap_means = np.log(bootstrap_means + 1e-30)  # Add a small epsilon to handle zeros
# variance = np.var(log_bootstrap_means)
# print(variance)


0.2799471240982541


In [101]:
bootstrap_means[0:200]

array([0.00083859, 0.00048591, 0.0003977 , 0.00045198, 0.00035291,
       0.00058272, 0.0003522 , 0.00050414, 0.00072202, 0.00080958,
       0.00050293, 0.00039444, 0.00036934, 0.00034041, 0.00042223,
       0.00018258, 0.00045941, 0.00100866, 0.00023944, 0.00025301,
       0.00067334, 0.00088244, 0.00043585, 0.00032923, 0.00046097,
       0.00069782, 0.00018635, 0.00061946, 0.00023485, 0.00022877,
       0.00079735, 0.00039855, 0.00046781, 0.00084626, 0.00035536,
       0.0001066 , 0.00081174, 0.00048926, 0.0003119 , 0.00042377,
       0.00042202, 0.00042292, 0.00036902, 0.00073654, 0.00048526,
       0.00070892, 0.00046643, 0.00048265, 0.00030032, 0.00054691,
       0.00084179, 0.0004032 , 0.00024322, 0.00065421, 0.00066153,
       0.00087003, 0.00046687, 0.0006341 , 0.00043848, 0.00062417,
       0.00050741, 0.00094303, 0.00079775, 0.00049285, 0.00042433,
       0.00072435, 0.00040096, 0.0006425 , 0.0007355 , 0.0006746 ,
       0.00034693, 0.00037134, 0.00069853, 0.00067243, 0.00074

In [84]:
np.var(bootstrap_means)

3.729209224658027e-08

In [85]:
np.var(normalized_bootstrap_means)

0.01785870756374992

In [ ]:
bootstrap_means

In [62]:
episode_probs_b_and_e[0]

[([2.0, 9.0], 3, 0.96, 0.36, 2.6666666666666665),
 ([1.0, 9.0], 3, 0.96, 0.36, 2.6666666666666665),
 ([0.0, 9.0], 1, 0.96, 0.36, 2.6666666666666665),
 ([0.0, 8.0], 4, 0.01, 0.16, 0.0625),
 ([1.0, 8.0], 0, 0.01, 0.16, 0.0625),
 ([1.0, 8.0], 1, 0.96, 0.36, 2.6666666666666665),
 ([1.0, 7.0], 3, 0.96, 0.36, 2.6666666666666665),
 ([0.0, 7.0], 1, 0.96, 0.36, 2.6666666666666665),
 ([0.0, 6.0], 1, 0.96, 0.36, 2.6666666666666665),
 ([0.0, 5.0], 1, 0.96, 0.36, 2.6666666666666665),
 ([0.0, 4.0], 4, 0.96, 0.36, 2.6666666666666665),
 ([1.0, 4.0], 0, 0.01, 0.16, 0.0625),
 ([1.0, 4.0], 2, 0.01, 0.16, 0.0625),
 ([1.0, 4.0], 1, 0.96, 0.36, 2.6666666666666665),
 ([1.0, 3.0], 3, 0.01, 0.16, 0.0625),
 ([0.0, 3.0], 4, 0.01, 0.16, 0.0625),
 ([1.0, 3.0], 4, 0.96, 0.36, 2.6666666666666665),
 ([2.0, 3.0], 4, 0.01, 0.16, 0.0625),
 ([3.0, 3.0], 1, 0.96, 0.36, 2.6666666666666665),
 ([3.0, 2.0], 1, 0.96, 0.36, 2.6666666666666665),
 ([3.0, 1.0], 3, 0.01, 0.16, 0.0625),
 ([2.0, 1.0], 0, 0.01, 0.16, 0.0625),
 ([2.0, 

In [103]:
pi_b[0]

array([([2., 9.], 0, 0., [2., 9.],  0, 0.00223088),
       ([2., 9.], 3, 0., [1., 9.],  1, 0.00223088),
       ([1., 9.], 1, 0., [1., 8.],  2, 0.00606415),
       ([1., 8.], 3, 0., [0., 8.],  3, 0.01648407),
       ([0., 8.], 0, 0., [0., 8.],  4, 0.04480836),
       ([0., 8.], 2, 0., [0., 9.],  5, 0.04480836),
       ([0., 9.], 1, 0., [0., 8.],  6, 0.01648407),
       ([0., 8.], 2, 0., [0., 9.],  7, 0.04480836),
       ([0., 9.], 0, 0., [0., 9.],  8, 0.01648407),
       ([0., 9.], 1, 0., [0., 8.],  9, 0.01648407),
       ([0., 8.], 0, 0., [0., 8.], 10, 0.04480836),
       ([0., 8.], 1, 0., [0., 7.], 11, 0.04480836),
       ([0., 7.], 4, 0., [1., 7.], 12, 0.12180176),
       ([1., 7.], 0, 0., [1., 7.], 13, 0.04480836),
       ([1., 7.], 2, 0., [1., 8.], 14, 0.04480836),
       ([1., 8.], 1, 0., [1., 7.], 15, 0.01648407),
       ([1., 7.], 3, 0., [0., 7.], 16, 0.04480836),
       ([0., 7.], 4, 0., [1., 7.], 17, 0.12180176),
       ([1., 7.], 0, 0., [1., 7.], 18, 0.04480836),
       ([1.,

In [ ]:
np.sum(0.99**np.arange(len(rews)) * np.cumprod(np.array(rhos)) * np.array(rews))

In [106]:
rhos_0 = episode_rhos[0]

In [108]:
rhos_0

[1.411764705882353,
 1.411764705882353,
 1.411764705882353,
 1.411764705882353,
 1.411764705882353,
 1.411764705882353,
 1.411764705882353,
 1.411764705882353,
 1.411764705882353,
 1.411764705882353,
 1.411764705882353,
 1.411764705882353,
 1.411764705882353,
 1.411764705882353,
 1.411764705882353,
 0.125,
 1.411764705882353,
 1.411764705882353]

In [107]:
np.cumprod(np.array(rhos_0))

array([  1.41176471,   1.99307958,   2.81375941,   3.97236623,
         5.60804644,   7.91724204,  11.17728288,  15.77969348,
        22.27721432,  31.45018492,  44.40026106,  62.6827215 ,
        88.49325388, 124.93165254, 176.3740977 ,  22.04676221,
        31.12484077,  43.94095168])

In [105]:
episode_rews[0]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0]

In [ ]:
P_pi_e

array([[[0.01, 0.01, 0.01, 0.01, 0.96],
        [0.01, 0.01, 0.96, 0.01, 0.01],
        [0.01, 0.01, 0.96, 0.01, 0.01],
        [0.01, 0.01, 0.96, 0.01, 0.01],
        [0.01, 0.01, 0.01, 0.01, 0.96],
        [0.01, 0.96, 0.01, 0.01, 0.01],
        [0.01, 0.96, 0.01, 0.01, 0.01],
        [0.01, 0.96, 0.01, 0.01, 0.01],
        [0.01, 0.96, 0.01, 0.01, 0.01],
        [0.01, 0.96, 0.01, 0.01, 0.01]],

       [[0.01, 0.01, 0.01, 0.01, 0.96],
        [0.01, 0.01, 0.96, 0.01, 0.01],
        [0.01, 0.01, 0.96, 0.01, 0.01],
        [0.01, 0.01, 0.01, 0.01, 0.96],
        [0.01, 0.96, 0.01, 0.01, 0.01],
        [0.01, 0.01, 0.01, 0.01, 0.96],
        [0.01, 0.01, 0.01, 0.96, 0.01],
        [0.01, 0.01, 0.01, 0.96, 0.01],
        [0.01, 0.96, 0.01, 0.01, 0.01],
        [0.01, 0.01, 0.01, 0.96, 0.01]],

       [[0.01, 0.01, 0.01, 0.01, 0.96],
        [0.01, 0.01, 0.96, 0.01, 0.01],
        [0.01, 0.01, 0.01, 0.01, 0.96],
        [0.01, 0.96, 0.01, 0.01, 0.01],
        [0.01, 0.96, 0.01, 0.01, 0.0

In [ ]:
def kl_divergence(P, Q):
    return np.sum(P * np.log(P / Q))

def compute_kl_divergences(Q1, Q2):
    kl_divs = []
    for i in range(Q1.shape[0]):
        for j in range(Q1.shape[1]):
            P = Q1[i, j]
            Q = Q2[i, j]
            kl_divs.append(kl_divergence(P, Q))
    return kl_divs

In [ ]:
P_pi_e[0,1]

array([0.01, 0.01, 0.96, 0.01, 0.01])

In [ ]:
P_pi_b = test_experiment.action_probs_top_n_epsilon(2,0.4)
P_pi_e = test_experiment.action_probs_top_n_epsilon(1,0.05)

In [ ]:
# Compute KL divergences
kl_divs = compute_kl_divergences(P_pi_e, P_pi_b)

# Aggregate KL divergences
average_kl_divergence = np.mean(kl_divs)

In [ ]:
kl_divs

[0.7909324426241334,
 0.7909324426241333,
 0.7909324426241333,
 0.7909324426241333,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241333,
 0.7909324426241333,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241333,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426241334,
 0.7909324426

In [ ]:
average_kl_divergence

0.24786920517286704

In [ ]:
def PDIS(trajectories,p_e,p_b,negligible_states=[],period=float("inf")):
    E_G = 0
    scores=[]
    for i, trajectory in enumerate(trajectories):
        G=0
        for t in range(1,len(trajectory)+1):
            importance_prod = 1
            if p_e is None and p_b is None:
                for (s,a,r,rho) in trajectory[0:t]:
                    if s not in negligible_states:
                        importance_prod = importance_prod * rho
            else:
                for (s,a,r) in trajectory[0:t]:
                    if s not in negligible_states:
                        importance_prod = importance_prod * p_e[s][a]/p_b[s][a]
            # use the last r
            G += importance_prod * r
        E_G+=G
        if i > 0 and i % period == 0:
            scores.append(E_G/(i+1))
    scores.append(E_G / len(trajectories))
    print("PDIS ",scores[-1])
    return scores

In [ ]:
test_load.save_heatmap()

In [ ]:
test_experiment = SCOPE_experiment(**params)
test_experiment.run_experiment()

The file 600_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_8_0.2_0.001_1e-05_1e-05_1_1_70.pt already exists in the folder.


In [ ]:
def plot_heatmap(data, save_path='heatmap.png'):
    values = np.zeros((10, 10))
    for (x, y), value in data.items():
        values[y, x] = value

    # Flip the values horizontally
    values = np.flip(values, axis=1)

    # Create the heatmap using Matplotlib
    plt.figure(figsize=(8, 6))
    plt.imshow(values, cmap='viridis', origin='lower')
    plt.colorbar(label='Values')

    # Add labels and title
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Heatmap')
    plt.xticks(np.arange(10), labels=np.arange(10))
    plt.yticks(np.arange(10), labels=np.arange(9, -1, -1))

    # Save the plot
    plt.savefig(save_path)

    # Show the plot
    plt.show()

# Dummy data for testing
dummy_data = {(i, j): i * 10 + j for i in range(10) for j in range(10)}

# Test the plotting function
plot_heatmap(dummy_data, save_path='heatmap.png')

In [ ]:
# Define the varying trajectory lengths
num_trajectories = [200, 400, 600, 800, 1000]
# Define hidden_dims
hidden_dimens = [[8],[8,8],[32]]
# Define trajectory lengths
trajectory_length = [30, 50, 70, 100]

for num in num_trajectories:
  # Modify the base parameters for each experiment
  # params = base_params_test.copy()  # Make a copy to avoid modifying the base parameters
  params["num_trajectories"] = num  # Update the number of trajectories

  for dims in hidden_dimens:
    # params = base_params_test.copy()  # Make a copy to avoid modifying the base parameters
    params["hidden_dims"] = dims

    for len in trajectory_length:
      params["max_length"] = len

      # Create an instance of SCOPE_experiment with modified parameters
      test_experiment = SCOPE_experiment(**params)
      test_experiment.run_experiment()

Streaming output truncated to the last 5000 lines.
Total Loss: 3.4867157889696476
----------------------------------------
Epoch 187
IS mean: 0.42689932602545877,IS variance: 0.010250330599085356
SCOPE Var loss:  tensor(0.0886, dtype=torch.float64, grad_fn=<VarBackward0>)
MSE loss:  3.3721690147865973
SCOPE mean: 0.43945066292784213, SCOPE var: 0.01642347647810597
Total Loss: 3.4607559647653416
----------------------------------------
Epoch 188
IS mean: 0.42689932602545877,IS variance: 0.010250330599085356
SCOPE Var loss:  tensor(0.0881, dtype=torch.float64, grad_fn=<VarBackward0>)
MSE loss:  3.3468963191805345
SCOPE mean: 0.4399420037071746, SCOPE var: 0.016439998658329152
Total Loss: 3.434981902076833
----------------------------------------
Epoch 189
IS mean: 0.42689932602545877,IS variance: 0.010250330599085356
SCOPE Var loss:  tensor(0.0876, dtype=torch.float64, grad_fn=<VarBackward0>)
MSE loss:  3.32181137391798
SCOPE mean: 0.4402677806285472, SCOPE var: 0.01645277806321618
Total

In [ ]:
# Define the varying trajectory lengths
num_trajectories = [200, 400, 600, 800, 1000]
# Define hidden_dims
hidden_dimens = [[8],[8,8],[32]]
# Define trajectory lengths
trajectory_length = [30, 50, 70, 100]

for num in num_trajectories:
  # Modify the base parameters for each experiment
  # params = base_params_test.copy()  # Make a copy to avoid modifying the base parameters
  params["num_trajectories"] = num  # Update the number of trajectories

  for dims in hidden_dimens:
    # params = base_params_test.copy()  # Make a copy to avoid modifying the base parameters
    params["hidden_dims"] = dims

    for len in trajectory_length:
      params["max_length"] = len

      # Create an instance of SCOPE_experiment with modified parameters
      test_experiment = SCOPE_experiment(**params)
      # test_experiment.run_experiment()
      test_load = existing_experiments(test_experiment)
      # test_load.plot_metrics_save(save = True)
      print(test_load.experiment_instance.generate_file_name())
      # test_load.get_heatmap()
      test_load.save_heatmap()

200_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_0.2_0.001_1e-05_1e-05_1_1_30
200_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_0.2_0.001_1e-05_1e-05_1_1_50
200_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_0.2_0.001_1e-05_1e-05_1_1_70
200_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_0.2_0.001_1e-05_1e-05_1_1_100
200_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_8_0.2_0.001_1e-05_1e-05_1_1_30
200_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_8_0.2_0.001_1e-05_1e-05_1_1_50
200_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_8_0.2_0.001_1e-05_1e-05_1_1_70
200_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_8_0.2_0.001_1e-05_1e-05_1_1_100
200_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_32_0.2_0.001_1e-05_1e-05_1_1_30
200_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_32_0.2_0.001_1e-05_1e-05_1_1_50
200_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_32_0.2_0.001_1e-05_1e-05_1_1_70
200_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_32_0.2_0.001_1e-05_1e-05_1_1_100
400_0.99_0.3_

/content/Shaping/existing_experiments.py:272: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(8, 6))


400_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_32_0.2_0.001_1e-05_1e-05_1_1_30
400_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_32_0.2_0.001_1e-05_1e-05_1_1_50
400_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_32_0.2_0.001_1e-05_1e-05_1_1_70
400_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_32_0.2_0.001_1e-05_1e-05_1_1_100
600_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_0.2_0.001_1e-05_1e-05_1_1_30
600_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_0.2_0.001_1e-05_1e-05_1_1_50
600_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_0.2_0.001_1e-05_1e-05_1_1_70
600_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_0.2_0.001_1e-05_1e-05_1_1_100
600_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_8_0.2_0.001_1e-05_1e-05_1_1_30
600_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_8_0.2_0.001_1e-05_1e-05_1_1_50
600_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_8_0.2_0.001_1e-05_1e-05_1_1_70
600_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_8_0.2_0.001_1e-05_1e-05_1_1_100
600_0.99_0.3_

In [ ]:
num_trajectories = [200, 400, 600, 800, 1000]
dims = [[8], [8,8], [32]]
trajectory_length = [30, 50, 70, 100]

for i in dims:
  params["hidden_dims"] = i
  for len in trajectory_length:
    params["max_length"] = len
    viz_over_num_trajectories_multi(params, num_trajectories, save=True, folder_path=None, filename=None)

/content/Shaping/SCOPE_straight.py:115: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  padded_timestep_tensors = torch.tensor(padded_timesteps, dtype = self.dtype)


In [ ]:
# test_experiment.continue_training(200)

Epoch 1
IS mean: 0.3970366370089854,IS variance: 0.05239271808991322
SCOPE Var loss:  tensor(0.0951, dtype=torch.float64, grad_fn=<VarBackward0>)
MSE loss:  1.6409151562014548
SCOPE mean: -0.04567254602060098, SCOPE var: 0.03338151550439484
Total Loss: 1.7360354050523952
----------------------------------------
Epoch 2
IS mean: 0.3970366370089854,IS variance: 0.05239271808991322
SCOPE Var loss:  tensor(0.4372, dtype=torch.float64, grad_fn=<VarBackward0>)
MSE loss:  1.5705323031103025
SCOPE mean: -0.045928751469007593, SCOPE var: 0.03335577320105847
Total Loss: 2.007761324068151
----------------------------------------
Epoch 3
IS mean: 0.3970366370089854,IS variance: 0.05239271808991322
SCOPE Var loss:  tensor(0.4359, dtype=torch.float64, grad_fn=<VarBackward0>)
MSE loss:  1.5344446202269266
SCOPE mean: -0.042520794559055955, SCOPE var: 0.03330584676820575
Total Loss: 1.970296805517244
----------------------------------------
Epoch 4
IS mean: 0.3970366370089854,IS variance: 0.0523927180

In [ ]:
test_load = existing_experiments(test_experiment)

In [ ]:
test_load.get_heatmap()

In [ ]:
test_load.plot_metrics()

In [ ]:
env = test_experiment.initialize_env()

In [ ]:
env.main_deaths

[[9, 0],
 [9, 1],
 [9, 2],
 [9, 3],
 [9, 4],
 [9, 5],
 [9, 6],
 [9, 7],
 [9, 8],
 [9, 9],
 [8, 0]]

In [ ]:
env.dead_ends

[[5, 5],
 [5, 6],
 [5, 7],
 [5, 8],
 [5, 9],
 [6, 5],
 [6, 6],
 [6, 7],
 [6, 8],
 [6, 9],
 [7, 5],
 [7, 6],
 [7, 7],
 [7, 8],
 [7, 9],
 [8, 5],
 [8, 6],
 [8, 7],
 [8, 8],
 [8, 9]]

In [ ]:
env.dead_ends + env.main_deaths

[[5, 5],
 [5, 6],
 [5, 7],
 [5, 8],
 [5, 9],
 [6, 5],
 [6, 6],
 [6, 7],
 [6, 8],
 [6, 9],
 [7, 5],
 [7, 6],
 [7, 7],
 [7, 8],
 [7, 9],
 [8, 5],
 [8, 6],
 [8, 7],
 [8, 8],
 [8, 9],
 [9, 0],
 [9, 1],
 [9, 2],
 [9, 3],
 [9, 4],
 [9, 5],
 [9, 6],
 [9, 7],
 [9, 8],
 [9, 9],
 [8, 0]]

In [ ]:
def smallest_distance_to_dead_end_or_death(current_state,env):

    all_death = env.main_deaths + env.dead_ends
    return min(manhattan_distance(current_state, dead_end) for dead_end in all_death)





def bottleneck_four_regions_k_p9_a_1(current_state, env,k = 0.9,a = 1):

    # Bottom left region
    if current_state[0] <4 and current_state[1]>4:
        dist_to_bottleneck = manhattan_distance(current_state, [0,5])
        return k*math.exp(-a*dist_to_bottleneck)

    # Top left region
    elif current_state[0] <5 and current_state[1] <5:
        dist_to_recovery = smallest_distance_to_recovery(current_state,env)
        return k*math.exp(-a*dist_to_recovery) + 0.1

    # Top right region
    elif current_state[0]>4 and current_state[1]<5:
        dist_to_recovery = smallest_distance_to_recovery(current_state,env)
        dist_to_ded_death = smallest_distance_to_dead_end_or_death(current_state, env)
        # diff_distance = dist_to_recovery - dist_to_deadend
        return k*math.exp(-a*dist_to_recovery) - k*math.exp(-a*dist_to_ded_death)

    # Bottom right region
    elif current_state[0]>3 and current_state[1]>4:
        dist_to_ded_death = smallest_distance_to_dead_end_or_death(current_state, env)
        return -k*math.exp(-a*dist_to_ded_death) - 0.1

In [ ]:
def plot_bottleneck_function():
    grid_size = 10
    values = np.zeros((grid_size, grid_size))

    # Example environment
    # env = {
    #     'recovery': [[2, 2], [4, 4]],  # Define recovery points
    #     'deadend': [[7, 2], [9, 1]]   # Define deadend points
    # }

    for i in range(grid_size):
        for j in range(grid_size):
            current_state = [j, i]
            values[i, j] = bottleneck_four_regions_k_p9_a_1(current_state, env)

    # Reverse the y-axis values for the flip
    values = values[::-1]

    fig = go.Figure(data=go.Heatmap(
        z=values,
        x=list(range(grid_size)),
        y=list(range(grid_size)),
        colorscale='Viridis'
    ))

    fig.update_layout(
        title='Bottleneck Function Values on 10x10 Grid',
        xaxis_title='X',
        yaxis_title='Y',
        yaxis=dict(scaleanchor="x", scaleratio=1)  # Ensure the aspect ratio is equal
    )

    fig.show()

plot_bottleneck_function()

In [ ]:
grid_size = 10
values = np.zeros((grid_size, grid_size))

# # Example environment
# env = {
#     'recovery': [[2, 2], [4, 4]],  # Define recovery points
#     'deadend': [[7, 2], [9, 1]]   # Define deadend points
# }

for i in range(grid_size):
    for j in range(grid_size):
        current_state = [i, j]
        values[i, j] = bottleneck_four_regions_k_p9_a_1(current_state, env)

fig = go.Figure(data=go.Heatmap(
    z=values,
    x=list(range(grid_size)),
    y=list(range(grid_size)),
    colorscale='Viridis'
))

fig.update_layout(
    title='Bottleneck Function Values on 10x10 Grid',
    xaxis_title='X',
    yaxis_title='Y',
    yaxis=dict(scaleanchor="x", scaleratio=1)  # Ensure the aspect ratio is equal
)

fig.show()

NameError: name 'bottleneck_four_regions_k_p9_a_1' is not defined

# No dropout or regularization

## Combination of distance to dead end and recovery

## Misc.

In [ ]:
# Define the varying trajectory lengths
trajectory_lengths = [200, 400, 600, 800, 1000]

# Iterate over each trajectory length and create corresponding experiments
for length in trajectory_lengths:
    # Modify the base parameters for each experiment
    params = base_params_test.copy()  # Make a copy to avoid modifying the base parameters
    params["num_trajectories"] = length  # Update the number of trajectories

    # Create an instance of SCOPE_experiment with modified parameters
    test_experiment = SCOPE_experiment(**params)
    test_experiment.run_experiment()  # Assuming you have a run_experiment method

# Testing

In [ ]:
params = {
    # Parameters related to policy generation
    "pi_b_top_k": 1,
    "pi_b_epsilon": 0.4,
    "pi_e_top_k": 1,
    "pi_e_epsilon": 0.05,
    "q_table": q_table,
    "gamma": 0.99,
    "num_trajectories": 200,
    "num_bootstraps": 10000,
    "percent_to_estimate_phi": 0.3,

    # Parameters related to shaping
    "shaping_feature": smallest_distance_to_deadend,
    "shaping_coefficient": 0.1,

    # Parameters related to neural network architecture and training
    "hidden_dims": [8, 8],
    "learning_rate": 0.001,
    "dropout_prob": 0.2,
    "l1_reg": 0.00001,
    "l2_reg": 0.00001,
    "scope_weight": 1,
    "mse_weight": 1,
    "num_epochs": 300,

    # Parameters related to environment
    "max_length": 50,
    "death_drag": 0.0,
    # Other general parameters
    "dtype": torch.float64,
    "experiment_type": "test",
    "folder_path": "/content/drive/MyDrive/Lifegate_experiments"
    # "folder_path": "/content"
}

In [ ]:
test_experiment = SCOPE_experiment(**params)
test_load = existing_experiments(test_experiment)
# IS_all_means, IS_all_variances, Train_means, Train_variances, Test_means, Test_variances = test_load.load_multi_estimates()

In [ ]:
num_trajectories = [200, 400, 600, 800, 1000]
dims = [[8], [8,8], [32]]
trajectory_length = [30, 50, 70, 100]

for i in dims:
  params["hidden_dims"] = i
  for len in trajectory_length:
    params["max_length"] = len
    viz_over_num_trajectories_multi(params, num_trajectories, save=True, folder_path=None, filename=None)

/content/Shaping/multi_experiment_visualisations.py:203: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axs = plt.subplots(3, 1, figsize=(10, 18))


In [ ]:
num_trajectories = [200, 400, 600, 800, 1000]
viz_over_num_trajectories_multi(params, num_trajectories, save=True, folder_path=None, filename=None)

/content/Shaping/SCOPE_straight.py:115: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  padded_timestep_tensors = torch.tensor(padded_timesteps, dtype = self.dtype)


In [ ]:
test_load.get_heatmap()

In [ ]:
env = test_experiment.initialize_env()

In [ ]:
env.main_deaths

[[9, 0],
 [9, 1],
 [9, 2],
 [9, 3],
 [9, 4],
 [9, 5],
 [9, 6],
 [9, 7],
 [9, 8],
 [9, 9],
 [8, 0]]

In [ ]:
test_load.load_on_policy_estimate()

0.8514780808647641

In [ ]:
[estimate - test_load.load_on_policy_estimate() for estimate in IS_all_means]

[1.6832966869467842,
 1.7093622028177835,
 1.71117787960983,
 1.7293877726207971,
 1.7048175696304875]

In [ ]:
test_experiment.get_multi_experiment()

/content/Shaping/SCOPE_straight.py:115: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  padded_timestep_tensors = torch.tensor(padded_timesteps, dtype = self.dtype)


([2.5290068415280103,
  2.5482847820745445,
  2.553100612083396,
  2.5279662530334126,
  2.561294712012935],
 [2.2950895376523412,
  2.3045730505152293,
  2.25439504747676,
  2.354724263194598,
  2.301225143136116],
 [4.066361256809858,
  4.05700443802146,
  4.139420225588691,
  4.090003913832987,
  4.010852962560537],
 [16.255051428225073,
  16.039218957691634,
  16.407065601508204,
  16.61423409290501,
  15.986347081758947],
 [2.307795945640806,
  2.293187794614861,
  2.3104521327752665,
  2.2658604438947374,
  2.3152241242410154],
 [3.105943786183369,
  3.0943760957291935,
  3.082721648820779,
  3.0886605708466432,
  3.120175526084799])

# Continue Training

In [ ]:
base_params_load = {
    # Parameters related to policy generation
    "pi_b_top_k": 1,
    "pi_b_epsilon": 0.4,
    "pi_e_top_k": 1,
    "pi_e_epsilon": 0.05,
    "q_table": q_table,
    "gamma": 0.99,
    "num_trajectories": 200,
    "num_bootstraps": 10000,
    "percent_to_estimate_phi": 0.3,

    # Parameters related to shaping
    "shaping_feature": smallest_distance_to_deadend,
    "shaping_coefficient": 0.1,

    # Parameters related to neural network architecture and training
    "hidden_dims": [8],
    "learning_rate": 0.001,
    "dropout_prob": 0.2,
    "l1_reg": 0.00001,
    "l2_reg": 0.00001,
    "scope_weight": 1,
    "mse_weight": 1,
    "num_epochs": 300,

    # Parameters related to environment
    "max_length": 30,
    "death_drag": 0.0,
    # Other general parameters
    "dtype": torch.float64,
    "experiment_type": "test",
    "folder_path": "/content/drive/MyDrive/Lifegate_experiments"
    # "folder_path": "/content"
}

In [ ]:
test_experiment = SCOPE_experiment(**base_params_load)

In [ ]:
test_experiment.continue_training(100)

# Saving Plots

In [ ]:
base_params_load = {
    # Parameters related to policy generation
    "pi_b_top_k": 1,
    "pi_b_epsilon": 0.4,
    "pi_e_top_k": 1,
    "pi_e_epsilon": 0.05,
    "q_table": q_table,
    "gamma": 0.99,
    "num_trajectories": 200,
    "num_bootstraps": 10000,
    "percent_to_estimate_phi": 0.3,

    # Parameters related to shaping
    "shaping_feature": smallest_distance_to_deadend,
    "shaping_coefficient": 0.1,

    # Parameters related to neural network architecture and training
    "hidden_dims": [32],
    "learning_rate": 0.001,
    "dropout_prob": 0.2,
    "l1_reg": 0.00001,
    "l2_reg": 0.00001,
    "scope_weight": 1,
    "mse_weight": 1,
    "num_epochs": 300,

    # Parameters related to environment
    "max_length": 100,
    "death_drag": 0.0,
    # Other general parameters
    "dtype": torch.float64,
    "experiment_type": "test",
    "folder_path": "/content/drive/MyDrive/Lifegate_experiments"
    # "folder_path": "/content"
}

In [ ]:
num_trajectories = [200, 400, 600, 800, 1000]

def save__experiments_over_trajectories(base_params_load, num_trajectories):
  for i in num_trajectories:
    print("-"*100)
    print(f"{i} trajectories: ")
    print("-"*100)
    params = base_params_load.copy()
    params["num_trajectories"] = i
    test_experiment = SCOPE_experiment(**params)
    test_load = existing_experiments(test_experiment)
    test_load.plot_metrics_save(save = True)
    # test_load.plot_metrics()
    print(test_load.experiment_instance.generate_file_name())

  viz_over_num_trajectories_save(base_params_load, num_trajectories)

In [ ]:
num_trajectories = [200, 400, 600, 800, 1000]
dims = [[8], [8,8], [32]]


for i in dims:
  params_dim = base_params_load.copy()
  params_dim["hidden_dims"] = dims
  save__experiments_over_trajectories(params_dim, num_trajectories)


In [ ]:
num_trajectories = [200, 400, 600, 800, 1000]
viz_over_num_trajectories_save(base_params_load, num_trajectories)

## Latex figure code generation

In [ ]:
b = "123"
figure_names = ["a", "b"]
# LaTeX template for including figures
latex_template = r"""
\begin{figure}[h]
    \centering
    \includegraphics[width=\textwidth]{%s.png}
    \caption{Caption for %s}
    \label{fig:%s}
\end{figure}
"""

# Loop through figure names and generate LaTeX code
for name in figure_names:
    latex_code = latex_template % (name, b, name)
    print(latex_code)


\begin{figure}[h]
    \centering
    \includegraphics[width=\textwidth]{a.png}
    \caption{Caption for 123}
    \label{fig:a}
\end{figure}


\begin{figure}[h]
    \centering
    \includegraphics[width=\textwidth]{b.png}
    \caption{Caption for 123}
    \label{fig:b}
\end{figure}



In [ ]:
# Define the varying trajectory lengths
num_trajectories = [200, 400, 600, 800, 1000]
# Define hidden_dims
hidden_dimens = [[8],[8,8],[32]]
# Define trajectory lengths
trajectory_length = [30, 50, 70, 100]

for num in num_trajectories:
  # Modify the base parameters for each experiment
  # params = base_params_test.copy()  # Make a copy to avoid modifying the base parameters
  params["num_trajectories"] = num  # Update the number of trajectories

  for dims in hidden_dimens:
    # params = base_params_test.copy()  # Make a copy to avoid modifying the base parameters
    params["hidden_dims"] = dims

    for len in trajectory_length:
      params["max_length"] = len

      # Create an instance of SCOPE_experiment with modified parameters
      test_experiment = SCOPE_experiment(**params)
      # test_experiment.run_experiment()
      test_load = existing_experiments(test_experiment)
      # test_load.plot_metrics_save(save = True)
      # print(test_load.experiment_instance.generate_file_name())
      filename = test_load.experiment_instance.generate_file_name()

      num_trajectories = str(test_experiment.num_trajectories)
      hidden_dims = str(test_experiment.hidden_dims[0])
      max_length = str(test_experiment.max_length)

      latex_template = r"""
      \subsection*{Experiment with Trajectories: %s Hidden Dimensions: %s Max. Trajectory Length: %s}

      \begin{figure}[h]
          \centering
          \includegraphics[width=\textwidth]{plt_plots/%s.png}
          \caption{Metrics for Trajectories: %s Hidden Dimensions: %s Max. Trajectory Length: %s}
          \label{fig:%s}
      \end{figure}

      \begin{figure}[h]
          \centering
          \includegraphics[width=\textwidth]{plt_plots/shaping_heatmap_predictions_%s.png}
          \caption{Shaping prediction heatmap for with Trajectories: %s Hidden Dimensions: %s Max. Trajectory Length: %s}
          \label{fig:%s}
      \end{figure}
      """
      latex_code = latex_template % (num_trajectories, hidden_dims, max_length,filename, num_trajectories, hidden_dims, max_length,filename, filename, num_trajectories, hidden_dims, max_length,filename )
      print(latex_code)

      # test_load.get_heatmap()
      # test_load.save_heatmap()


      \subsection*{Experiment with Trajectories: 200 Hidden Dimensions: 8 Max. Trajectory Length: 30}

      \begin{figure}[h]
          \centering
          \includegraphics[width=\textwidth]{plt_plots/200_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_0.2_0.001_1e-05_1e-05_1_1_30.png}
          \caption{Metrics for Trajectories: 200 Hidden Dimensions: 8 Max. Trajectory Length: 30}
          \label{fig:200_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_0.2_0.001_1e-05_1e-05_1_1_30}
      \end{figure}

      \begin{figure}[h]
          \centering
          \includegraphics[width=\textwidth]{plt_plots/shaping_heatmap_predictions_200_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_0.2_0.001_1e-05_1e-05_1_1_30.png}
          \caption{Shaping prediction heatmap for with Trajectories: 200 Hidden Dimensions: 8 Max. Trajectory Length: 30}
          \label{fig:200_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_0.2_0.001_1e-05_1e-05_1_1_30}
      \end{figure}
      

      \subsection*{Expe

In [ ]:
# Define the varying trajectory lengths
num_trajectories_list = [200, 400, 600, 800, 1000]
# Define hidden_dims
hidden_dimens = [[8],[8,8],[32]]
# Define trajectory lengths
trajectory_length = [30, 50, 70, 100]

for dims in hidden_dimens:
  # params = base_params_test.copy()  # Make a copy to avoid modifying the base parameters
  params["hidden_dims"] = dims
  for len in trajectory_length:
    params["max_length"] = len
    for num in num_trajectories_list:
      params["num_trajectories"] = num  # Update the number of trajectories
      # Create an instance of SCOPE_experiment with modified parameters
      test_experiment = SCOPE_experiment(**params)
      # test_experiment.run_experiment()
      test_load = existing_experiments(test_experiment)
      # test_load.plot_metrics_save(save = True)
      # print(test_load.experiment_instance.generate_file_name())
      filename = test_load.experiment_instance.generate_file_name()

      num_trajectories = str(test_experiment.num_trajectories)
      hidden_dims = str(test_experiment.hidden_dims[0])
      max_length = str(test_experiment.max_length)

      latex_template = r"""
      \subsection*{Experiment with Trajectories: %s Hidden Dimensions: %s Max. Trajectory Length: %s}

      \begin{figure}[H]
          \centering
          \includegraphics[width=\textwidth]{plt_plots/%s.png}
          \caption{Metrics for Trajectories: %s Hidden Dimensions: %s Max. Trajectory Length: %s}
          \label{fig:%s}
      \end{figure}

      \begin{figure}[h]
          \centering
          \includegraphics[width=\textwidth]{plt_plots/shaping_heatmap_predictions_%s.png}
          \caption{Shaping prediction heatmap for with Trajectories: %s Hidden Dimensions: %s Max. Trajectory Length: %s}
          \label{fig:%s}
      \end{figure}
      """
      latex_code = latex_template % (num_trajectories, hidden_dims, max_length,filename, num_trajectories, hidden_dims, max_length,filename, filename, num_trajectories, hidden_dims, max_length,filename )
      print(latex_code)

      # test_load.get_heatmap()
      # test_load.save_heatmap()


      \subsection*{Experiment with Trajectories: 200 Hidden Dimensions: 8 Max. Trajectory Length: 30}

      \begin{figure}[h]
          \centering
          \includegraphics[width=\textwidth]{plt_plots/200_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_0.2_0.001_1e-05_1e-05_1_1_30.png}
          \caption{Metrics for Trajectories: 200 Hidden Dimensions: 8 Max. Trajectory Length: 30}
          \label{fig:200_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_0.2_0.001_1e-05_1e-05_1_1_30}
      \end{figure}

      \begin{figure}[h]
          \centering
          \includegraphics[width=\textwidth]{plt_plots/shaping_heatmap_predictions_200_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_0.2_0.001_1e-05_1e-05_1_1_30.png}
          \caption{Shaping prediction heatmap for with Trajectories: 200 Hidden Dimensions: 8 Max. Trajectory Length: 30}
          \label{fig:200_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_0.2_0.001_1e-05_1e-05_1_1_30}
      \end{figure}
      

      \subsection*{Expe

In [ ]:
for num in num_trajectories:
      params["num_trajectories"] = num
      print(params["num_trajectories"])

0


In [ ]:
num_trajectories

'0'

In [ ]:
# Define the varying trajectory lengths
num_trajectories_list = [200, 400, 600, 800, 1000]
# Define hidden_dims
hidden_dimens = [[8],[8,8],[32]]
# Define trajectory lengths
trajectory_length = [30, 50, 70, 100]

# for num in num_trajectories:
for dims in hidden_dimens:
  # Modify the base parameters for each experiment
  # params = base_params_test.copy()  # Make a copy to avoid modifying the base parameters
  # params["num_trajectories"] = num  # Update the number of trajectories
  params["hidden_dims"] = dims
  # for dims in hidden_dimens:
  for len in trajectory_length:
    # params = base_params_test.copy()  # Make a copy to avoid modifying the base parameters
    # params["hidden_dims"] = dims
    params['max_length'] = len

    # for len in trajectory_length:
    latex_subsection = r"""
    \subsection*{Experiment set with Hidden Dimensions: %s Max. Trajectory Length: %s}
    """
    # latex_sub =
    latex_sub = latex_subsection % (str(dims), str(len))
    print(latex_sub)
    for num in num_trajectories_list:
      # params["max_length"] = len
      params["num_trajectories"] = num

      # Create an instance of SCOPE_experiment with modified parameters
      test_experiment = SCOPE_experiment(**params)
      # test_experiment.run_experiment()
      test_load = existing_experiments(test_experiment)
      # test_load.plot_metrics_save(save = True)
      # print(test_load.experiment_instance.generate_file_name())
      filename = test_load.experiment_instance.generate_file_name()

      num_trajectories = str(test_experiment.num_trajectories)
      # print(num)
      # print(num_trajectories)
      hidden_dims = str(test_experiment.hidden_dims)
      max_length = str(test_experiment.max_length)

      latex_template = r"""
      \subsubsection*{Experiment with Trajectories: %s Hidden Dimensions: %s Max. Trajectory Length: %s}

      \begin{figure}[H]
          \centering
          \includegraphics[width=0.55\textwidth]{plt_plots/%s.png}
          \caption{Metrics for Trajectories: %s Hidden Dimensions: %s Max. Trajectory Length: %s}
          \label{fig:%s}
      \end{figure}

      \begin{figure}[H]
          \centering
          \includegraphics[width=0.55\textwidth]{plt_plots/shaping_heatmap_predictions_%s.png}
          \caption{Shaping prediction heatmap for with Trajectories: %s Hidden Dimensions: %s Max. Trajectory Length: %s}
          \label{fig:%s}
      \end{figure}
      """
      latex_code = latex_template % (num_trajectories, hidden_dims, max_length,filename, num_trajectories, hidden_dims, max_length,filename, filename, num_trajectories, hidden_dims, max_length,filename )
      print(latex_code)

      # test_load.get_heatmap()
      # test_load.save_heatmap()


    \subsection*{Experiment set with Hidden Dimensions: [8] Max. Trajectory Length: 30}
    
      
      \subsubsection*{Experiment with Trajectories: 200 Hidden Dimensions: [8] Max. Trajectory Length: 30}

      \begin{figure}[H]
          \centering
          \includegraphics[width=0.55\textwidth]{plt_plots/200_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_0.2_0.001_1e-05_1e-05_1_1_30.png}
          \caption{Metrics for Trajectories: 200 Hidden Dimensions: [8] Max. Trajectory Length: 30}
          \label{fig:200_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_0.2_0.001_1e-05_1e-05_1_1_30}
      \end{figure}

      \begin{figure}[H]
          \centering
          \includegraphics[width=0.55\textwidth]{plt_plots/shaping_heatmap_predictions_200_0.99_0.3_bottleneck_four_regions_k_p9_a_1_0.1_8_0.2_0.001_1e-05_1e-05_1_1_30.png}
          \caption{Shaping prediction heatmap for with Trajectories: 200 Hidden Dimensions: [8] Max. Trajectory Length: 30}
          \label{fig:200_0.99_0.3_bo

In [ ]:
str([8,8])

'[8, 8]'

In [ ]:
params

{'pi_b_top_k': 1,
 'pi_b_epsilon': 0.4,
 'pi_e_top_k': 1,
 'pi_e_epsilon': 0.05,
 'q_table': array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,

In [ ]:
# Define the varying trajectory lengths
num_trajectories = [200, 400, 600, 800, 1000]
# Define hidden_dims
hidden_dimens = [[8], [8, 8], [32]]
# Define trajectory lengths
trajectory_length = [30, 50, 70, 100]

# Base parameters (assuming base_params_test is defined somewhere in your code)
base_params_test = {}

# Placeholder for the LaTeX content
latex_content = r"""
\documentclass{article}
\usepackage{graphicx}
\begin{document}
"""

# Loop through the parameter combinations
for section_idx, num in enumerate(num_trajectories, start=1):
    # params = base_params_test.copy()  # Make a copy to avoid modifying the base parameters
    params["num_trajectories"] = num  # Update the number of trajectories

    for dims in hidden_dimens:
        params["hidden_dims"] = dims

        for length in trajectory_length:
            params["max_length"] = length

            # Create an instance of SCOPE_experiment with modified parameters
            test_experiment = SCOPE_experiment(**params)
            # test_experiment.run_experiment()
            test_load = existing_experiments(test_experiment)
            # test_load.plot_metrics_save(save=True)

            # Generate file name from the experiment instance
            file_name = test_load.experiment_instance.generate_file_name()

            # Convert parameters to string
            num_trajectories_str = str(test_experiment.num_trajectories)
            hidden_dims_str = '-'.join(map(str, test_experiment.hidden_dims))
            max_length_str = str(test_experiment.max_length)

            # Format the LaTeX code
            latex_code = r"""
\subsection*{Section %d: Trajectories=%s, HiddenDims=%s, MaxLength=%s}

\begin{figure}[h]
    \centering
    \includegraphics[width=\textwidth]{%s.png}
    \caption{Metrics for Trajectories=%s, HiddenDims=%s, MaxLength=%s}
    \label{fig:%s}
\end{figure}

\begin{figure}[h]
    \centering
    \includegraphics[width=\textwidth]{shaping_heatmap_predictions_%s.png}
    \caption{Shaping prediction heatmap for Trajectories=%s, HiddenDims=%s, MaxLength=%s}
    \label{fig:shaping_%s}
\end{figure}
""" % (section_idx, num_trajectories_str, hidden_dims_str, max_length_str,
       file_name, num_trajectories_str, hidden_dims_str, max_length_str, file_name,
       num_trajectories_str, hidden_dims_str, max_length_str, num_trajectories_str)

            # Print the LaTeX code for the current iteration
            print(latex_code)

            # Append the LaTeX code to the content
            latex_content += latex_code

            # Generate and save the heatmap
            test_load.save_heatmap()

# End of LaTeX document content
latex_content += r"""
\end{document}
"""

# Print the complete LaTeX document content
print(latex_content)


TypeError: not enough arguments for format string

In [ ]:
str(test_experiment.max_length)

'100'

# Clean Loading

In [ ]:
# Loading metrics


# Experiments Run:

### Define the varying trajectory lengths
num_trajectories = [200, 400, 600, 800, 1000]
### Define hidden_dims
hidden_dimens = [[8],[8,8],[32]]
### Define trajectory lengths
trajectory_length = [30, 50, 70, 100]



In [ ]:
base_params_load = {
    # Parameters related to policy generation
    "pi_b_top_k": 1,
    "pi_b_epsilon": 0.4,
    "pi_e_top_k": 1,
    "pi_e_epsilon": 0.05,
    "q_table": q_table,
    "gamma": 0.99,
    "num_trajectories": 200,
    "num_bootstraps": 10000,
    "percent_to_estimate_phi": 0.3,

    # Parameters related to shaping
    "shaping_feature": smallest_distance_to_deadend,
    "shaping_coefficient": 0.1,

    # Parameters related to neural network architecture and training
    "hidden_dims": [8],
    "learning_rate": 0.001,
    "dropout_prob": 0.2,
    "l1_reg": 0.00001,
    "l2_reg": 0.00001,
    "scope_weight": 1,
    "mse_weight": 1,
    "num_epochs": 300,

    # Parameters related to environment
    "max_length": 50,
    "death_drag": 0.0,
    # Other general parameters
    "dtype": torch.float64,
    "experiment_type": "test",
    "folder_path": "/content/drive/MyDrive/Lifegate_experiments"
    # "folder_path": "/content"
}

## Individual experiments

In [ ]:
test_experiment = SCOPE_experiment(**base_params_load)
test_load = existing_experiments(test_experiment)

In [ ]:
!pip install -U kaleido

In [ ]:
import kaleido


In [ ]:
test_load.plot_metrics_save(save = True)

In [ ]:
!pip install psutil


In [ ]:
import plotly.graph_objects as go
import os

# Create a simple plot
fig = go.Figure(data=go.Bar(y=[2, 3, 1]))

# Define a save path (ensure the directory exists and is writable)
# save_path = os.path.join(os.getcwd(), "simple_plot.png")

# Save the figure using orca
try:
    fig.write_image("simple_plot.png", engine="orca")
    # print(f"Image saved successfully at {save_path}")
except Exception as e:
    print(f"Error saving image: {e}")

# Display the figure
fig.show()

In [ ]:
test_load.plot_metrics_save(save = True)

In [ ]:
test_load.plot_metrics()

In [ ]:
print(test_load.experiment_instance.generate_file_name())

In [ ]:
file_test = test_load.experiment_instance.generate_file_name()

In [ ]:
file_test

In [ ]:
"over_trajectories_" + file_test.split('_', 1)[1]

In [ ]:
num_trajectories = [200, 400, 600, 800, 1000]

for i in num_trajectories:
  print("-"*100)
  print(f"{i} trajectories: ")
  print("-"*100)
  params = base_params_load.copy()
  params["num_trajectories"] = i
  test_experiment = SCOPE_experiment(**params)
  test_load = existing_experiments(test_experiment)
  test_load.plot_metrics_save(save = True)
  # test_load.plot_metrics()
  # print(test_load.experiment_instance.generate_file_name())



In [ ]:
test_load.get_state_visitation_heatmap()

In [ ]:
test_load.get_heatmap()

## Multiple experiments

In [ ]:
base_params_load = {
    # Parameters related to policy generation
    "pi_b_top_k": 1,
    "pi_b_epsilon": 0.4,
    "pi_e_top_k": 1,
    "pi_e_epsilon": 0.05,
    "q_table": q_table,
    "gamma": 0.99,
    "num_trajectories": 1000,
    "num_bootstraps": 10000,
    "percent_to_estimate_phi": 0.3,

    # Parameters related to shaping
    "shaping_feature": smallest_distance_to_deadend,
    "shaping_coefficient": 0.1,

    # Parameters related to neural network architecture and training
    "hidden_dims": [8],
    "learning_rate": 0.001,
    "dropout_prob": 0.2,
    "l1_reg": 0.00001,
    "l2_reg": 0.00001,
    "scope_weight": 1,
    "mse_weight": 1,
    "num_epochs": 300,

    # Parameters related to environment
    "max_length": 100,
    "death_drag": 0.0,
    # Other general parameters
    "dtype": torch.float64,
    "experiment_type": "test",
    "folder_path": "/content/drive/MyDrive/Lifegate_experiments"
    # "folder_path": "/content"
}

### Over num trajectories

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
num_trajectories = [200, 400, 600, 800, 1000]
viz_over_num_trajectories_save(base_params_load, num_trajectories)

### Over train set size

In [ ]:
train_set_sizes = [0.1, 0.2, 0.3, 0.4, 0.5]
viz_over_train_set(base_params_load, train_set_sizes)